In [1]:
import mysql.connector
from functools import partial


def get_connection_helper():

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host='localhost',
        user='megadados',
        password='megadados',
        database='XPTO',
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

a) Liste as 3 pizzas que dão mais lucro. O lucro de uma pizza é o seu preço de venda menos o 
custo total dos ingredientes nela utilizados (não se esqueça de que cada ingrediente da pizza tem uma 
especificação de quantidade a ser utilizada)

In [83]:
db('''DROP TABLE IF EXISTS custo''')
db('''
CREATE TEMPORARY TABLE custo
SELECT
    pizza, SUM(preco_unitario * quantidade) as custo
FROM
    pizza
    INNER JOIN ingredientepizza USING (pizza)
    INNER JOIN ingrediente USING (ingrediente)
GROUP BY
    pizza
''')

Executando query:
Executando query:


In [84]:
db('''
SELECT
    *
FROM
    custo
''')

Executando query:
('calabresa', 95.0)
('incoerente', 117.0)
('legumes', 22.0)
('mussarela', 35.0)
('pão', 5.0)


In [85]:
db('''
SELECT
    pizza, (preco - custo) as lucro
FROM
    pizza
    INNER JOIN custo USING (pizza)
ORDER BY
    lucro DESC
LIMIT
    3
''')

Executando query:
('pão', 95.0)
('incoerente', 83.0)
('legumes', 78.0)


b) Construa uma tabela temporária que liste todas as pizzas e a quantidade de ingredientes nela

In [86]:
db('''DROP TABLE IF EXISTS pizza_quantidade_ingredientes''')

Executando query:


In [87]:
db('''
CREATE TEMPORARY TABLE pizza_quantidade_ingredientes
SELECT
    pizza, COUNT(ingrediente) as quantidade_ingredientes
FROM
    pizza
    INNER JOIN ingredientepizza USING (pizza)
    INNER JOIN ingrediente USING (ingrediente)
GROUP BY
    pizza
''')

Executando query:


In [88]:
db('''
SELECT 
    *
FROM
    pizza_quantidade_ingredientes
''')

Executando query:
('calabresa', 2)
('incoerente', 4)
('legumes', 2)
('mussarela', 2)
('pão', 1)


c) Construa uma tabela temporária que liste todas as pizzas a quantidade de ingredientes
veganos nela (ou seja, onde o tipo é “vegana”)

In [89]:
db('''DROP TABLE IF EXISTS pizza_quantidade_ingredientes_veganos''')
db('''
CREATE TEMPORARY TABLE pizza_quantidade_ingredientes_veganos
SELECT
    pizza, COUNT(ingrediente) as quantidade_vegano
FROM
    pizza
    INNER JOIN ingredientepizza USING (pizza)
    INNER JOIN ingrediente USING (ingrediente)
    INNER JOIN ingredientetipo USING (ingrediente)
    INNER JOIN tipo USING (tipo)
WHERE
    tipo = 'vegana'
GROUP BY
    pizza
''')

db('''
SELECT
    *
FROM
    pizza_quantidade_ingredientes_veganos
''')

Executando query:
Executando query:
Executando query:
('legumes', 2)
('incoerente', 2)


d) Usando as tabelas temporárias dos itens (b) e (c) – mesmo que você não os tenha feito – liste
as pizzas veganas da pizzaria. Uma pizza é vegana se todos os seus ingredientes são veganos.

In [90]:
db('''
SELECT
    *
FROM
    pizza_quantidade_ingredientes
    INNER JOIN pizza_quantidade_ingredientes_veganos USING (pizza)
WHERE 
    quantidade_vegano = quantidade_ingredientes
''')

Executando query:
('legumes', 2, 2)


e) Liste os ingredientes e seus tipos, um ingrediente por linha (Dica: use a função
GROUP_CONCAT). O resultado deve ser da seguinte forma:

In [91]:
db('''
SELECT
    ingrediente, GROUP_CONCAT(tipo) as lista_tipos 
FROM
    ingrediente
    INNER JOIN ingredientetipo USING (ingrediente)
    INNER JOIN tipo USING (tipo)
GROUP BY
    ingrediente
''')

Executando query:
('abobrinha', 'sem gluten,sem lactose,vegana,vegetariana')
('calabresa', 'sem gluten,sem lactose')
('massa', 'vegetariana')
('massa sem gluten', 'sem gluten,sem lactose,vegana,vegetariana')
('mussarela', 'sem gluten,vegetariana')
('mussarela vegana', 'sem gluten,sem lactose,vegana,vegetariana')
('picanha', 'sem gluten,sem lactose')
